# Train an anti-aliased U-net based GAN

## Imports and GPU identification

In [ ]:
import tensorflow as tf
import os
import time
from PIL import Image
from scipy import special
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.python.client import device_lib
from datetime import datetime

print(device_lib.list_local_devices())
print(tf.config.experimental.get_memory_info("/device:GPU:0")['current'])

## Test Keras convolution
Find out what the Keras convolution automatic zero paddings are for padding = "SAME", depending on the input size and kernel size. For skip connections it is important that the data stays centered spatially. These tests need not be run for the rest of the notebook to work.

In [ ]:
print("Downsampling, even size input, even size kernel:")
print("Input:")
input_ = tf.ones((1, 8, 8, 1), dtype=tf.float32)
print(input_[0, :, :, 0].numpy())
print("Kernel:")
kernel = tf.ones((4, 4, 1, 1), dtype=tf.float32)
print(kernel[:, :, 0, 0].numpy())
print("Output of strides = 2, padding = 'SAME' convolution:")
conv = tf.nn.conv2d(input = input_, filters = kernel, strides = 2, padding = 'SAME')
print(conv[0, :, :, 0].numpy())
print("The output is properly centered.")
print("Output size = input size / 2")

print("\nDownsampling, odd size input, even size kernel:")
print("Input:")
input_ = tf.ones((1, 7, 7, 1), dtype=tf.float32)
print(input_[0, :, :, 0].numpy())
print("Kernel:")
kernel = tf.ones((4, 4, 1, 1), dtype=tf.float32)
print(kernel[:, :, 0, 0].numpy())
print("Output of strides = 2, padding = 'SAME' convolution:")
conv = tf.nn.conv2d(input = input_, filters = kernel, strides = 2, padding = 'SAME')
print(conv[0, :, :, 0].numpy())
print("Invalid output centering! Don't do this.")

print("\nDownsampling, even size input, odd size kernel:")
print("Input:")
input_ = tf.ones((1, 8, 8, 1), dtype=tf.float32)
print(input_[0, :, :, 0].numpy())
print("Kernel:")
kernel = tf.ones((3, 3, 1, 1), dtype=tf.float32)
print(kernel[:, :, 0, 0].numpy())
print("Output of strides = 2, padding = 'SAME' convolution:")
conv = tf.nn.conv2d(input = input_, filters = kernel, strides = 1, padding = 'SAME')
print(conv[0, :, :, 0].numpy())
print("Invalid output centering! Don't do this.")

print("\nDownsampling, odd size input, odd size kernel:")
print("Input:")
input_ = tf.ones((1, 9, 9, 1), dtype=tf.float32)
print(input_[0, :, :, 0].numpy())
print("Kernel:")
kernel = tf.ones((5, 5, 1, 1), dtype=tf.float32)
print(kernel[:, :, 0, 0].numpy())
print("Output of strides = 2, padding = 'SAME' convolution:")
conv = tf.nn.conv2d(input = input_, filters = kernel, strides = 1, padding = 'SAME')
print(conv[0, :, :, 0].numpy())
print("The output is properly centered.")
print("Output size = (input size + 1) / 2")

In [ ]:
print("Upsampling, any size input, even size kernel:")
print("Input:")
input_ = tf.ones((1, 4, 4, 1), dtype=tf.float32)
print(input_[0, :, :, 0].numpy())
tile = tf.constant([[0, 0], [0, 4]], dtype=tf.float32)
input_shape = input_.get_shape()
tiles = tf.reshape(tf.tile(tile, [input_shape[-3], input_shape[-2]]), (1, input_shape[-3]*2, input_shape[-2]*2, 1))
print("Diluted:")
diluted = tf.keras.layers.UpSampling2D(size = (2, 2), interpolation='nearest')(input_)*tiles
print(diluted[0, :, :, 0].numpy());
print("Kernel:")
kernel = tf.ones((4, 4, 1, 1), dtype=tf.float32)
print(kernel[:, :, 0, 0].numpy())
print("Output of strides = 1, padding = 'SAME' convolution:")
conv = tf.nn.conv2d(input = diluted, filters = kernel, strides = 1, padding = 'SAME')
print(conv[0, :, :, 0].numpy())
print("The output is properly centered.")
print("Output size = input size * 2")

print("\nUpsampling, any size input, odd size kernel:")
print("Input:")
input_ = tf.ones((1, 4, 4, 1), dtype=tf.float32)
print(input_[0, :, :, 0].numpy())
tile = tf.constant([[0, 0], [0, 4]], dtype=tf.float32)
input_shape = input_.get_shape()
tiles = tf.reshape(tf.tile(tile, [input_shape[-3], input_shape[-2]])[1:,1:], (1, input_shape[-3]*2 - 1, input_shape[-2]*2 - 1, 1))
print("Diluted:")
diluted = tf.keras.layers.UpSampling2D(size = (2, 2), interpolation='nearest')(input_)[:,1:,1:,:]*tiles
print(diluted[0, :, :, 0].numpy());
print("Kernel:")
kernel = tf.ones((3, 3, 1, 1), dtype=tf.float32)
print(kernel[:, :, 0, 0].numpy())
print("Output of strides = 1, padding = 'SAME' convolution:")
conv = tf.nn.conv2d(input = diluted, filters = kernel, strides = 1, padding = 'SAME')
print(conv[0, :, :, 0].numpy())
print("The output is properly centered.")
print("Output size = input size * 2 - 1")


## Circularly symmetric low-pass or anti-aliasing filter
Functions for generating a filter kernel with a parameterized cutoff frequency and kernel size

In [ ]:
def circular_lowpass_kernel(omega_c, N):  # omega = cutoff frequency in radians (pi is max), N = horizontal size of the kernel, also its vertical size.
    with np.errstate(divide='ignore',invalid='ignore'):    
        kernel = np.fromfunction(lambda x, y: omega_c*special.j1(omega_c*np.sqrt((x - (N - 1)/2)**2 + (y - (N - 1)/2)**2))/(2*np.pi*np.sqrt((x - (N - 1)/2)**2 + (y - (N - 1)/2)**2)), [N, N])
    if N % 2:
        kernel[(N - 1)//2, (N - 1)//2] = omega_c**2/(4*np.pi)
    return kernel

def rotated_cosine_window(N):  # N = horizontal size of the targeted kernel, also its vertical size, must be odd.
    return np.fromfunction(lambda y, x: np.maximum(np.cos(np.pi/2*np.sqrt(((x - (N - 1)/2)/((N - 1)/2 + 1))**2 + ((y - (N - 1)/2)/((N - 1)/2 + 1))**2)), 0), [N, N])

def windowed_circular_lowpass_kernel(cutoff, kernel_size):
    return (circular_lowpass_kernel(cutoff, kernel_size)*rotated_cosine_window(kernel_size)).astype(np.float32);

def test_kernel(kernel):
    kernel_size = kernel.shape[0] #height, should be same as width
    
    print("Kernel:")
    plt.imshow(kernel, vmin=-0.25, vmax=0.25, cmap='bwr', extent=[-kernel_size/2, kernel_size/2, -kernel_size/2, kernel_size/2])
    plt.colorbar()
    plt.show()

    print("Frequency response:")
    freq_resp = np.abs(np.fft.fftshift(np.fft.fft2(np.roll(np.pad(kernel, 101, mode = 'constant', constant_values = 0), shift = (-kernel_size//2-99, -kernel_size//2-99), axis = (0, 1)))))
    plt.imshow(freq_resp, vmin=-1.2, vmax=1.2, cmap='bwr', extent=[-np.pi, np.pi, -np.pi, np.pi])
    plt.colorbar()
    plt.show()

## Load training target images and low-pass filter them

Note: Do NOT use the filtered PNG images as network input, because they have been quantized. They can be saved for visual inspection.

In [ ]:
# Configuration
oversampling = 2  # Oversampling factor, 2 is a good choice
aa_even_kernel_size = 12 # Size of even sized antialiasing filter
aa_odd_kernel_size = aa_even_kernel_size + 1 # Size of odd sized antialiasing filter
approx_batch_size = 4
train_image_cropped_size = 256
photos_dir = "C:/Users/romat/Desktop/anotherfolder"
save_filtered_photos_dir = "D:/roman/filtered_grass_images_subset" # save folder, or None to disable saving

print("Even size kernel:")
test_kernel(windowed_circular_lowpass_kernel(np.pi/oversampling, aa_even_kernel_size))
print("Odd size kernel:")
test_kernel(windowed_circular_lowpass_kernel(np.pi/oversampling, aa_odd_kernel_size))

def channeled_kernel(kernel_prototype, num_channels):
    kernel_expanded = tf.expand_dims(kernel_prototype, 2)
    kernel_repeated = tf.repeat(kernel_expanded, num_channels, axis=2)
    return tf.expand_dims(kernel_repeated, -1)

def load_and_filter_image(filepath, channeled_kernel):
    image = tf.io.decode_png(tf.io.read_file(filepath), channels=3, dtype=tf.dtypes.uint8)
    image = tf.image.convert_image_dtype(image, dtype=tf.float32) * 2 - 1
    image = tf.nn.depthwise_conv2d([image], channeled_kernel, [1, 1, 1, 1], "VALID")[0]
    if save_filtered_photos_dir is not None:
        basename = tf.py_function(lambda string_tensor: os.path.basename(string_tensor.numpy()), [filepath], tf.string)
        tf.io.write_file(save_filtered_photos_dir + "/" + basename, tf.io.encode_png(tf.image.convert_image_dtype((image + 1)/2, dtype=tf.dtypes.uint8, saturate=True)))
    return image

train_dataset = tf.data.Dataset.list_files("C:/Users/romat/Desktop/anotherfolder/*.png")
print(train_dataset)
train_dataset = train_dataset.map(lambda filepath: load_and_filter_image(filepath, channeled_kernel(windowed_circular_lowpass_kernel(np.pi/oversampling, aa_even_kernel_size), 3)), num_parallel_calls=tf.data.experimental.AUTOTUNE, deterministic=False)
train_dataset = train_dataset.cache() # Load photos now instead of doing it again for each copy made in the next line
train_dataset = train_dataset.repeat(tf.math.floordiv(approx_batch_size, train_dataset.cardinality()))
train_dataset = train_dataset.cache()

batch_size = train_dataset.cardinality() # batch size
print("Batch size = " + str(batch_size.numpy()))

## Isotropic lowpass filtering layers
These layers apply a circularly symmmetrical lowpass filter. They preserve spatial centering of the data to facilitate skip connections and center cropping in the network.

`IsotropicLowpassFilterLayer` accepts `strides = 1` to preserve the feature map size and `strides = 2` to downsample it to half size. With `strides = 1`, the kernel size must be odd. With downsampling, only even input and kernel sizes are supported, resulting in even-sized output. `cutoff` is the cutoff frequency (before downsampling).

`IsotropicLowpassFilteredUpsampleLayer` doubles the spatial size of the feature map. `cutoff` is the cutoff frequency after upsampling. `kernel_size` must be even.

In [ ]:
class IsotropicLowpassFilterLayer(tf.keras.layers.Layer):
    def __init__(self, cutoff, kernel_size, strides = 1):
        super(IsotropicLowpassFilterLayer, self).__init__()
        self.kernel_size = kernel_size
        self.strides = strides        
        assert self.strides == 1 or self.strides == 2, "Accepted values for strides are 1 and 2. Tuples or arrays are not accepted."
        if self.strides == 1:
            assert self.kernel_size % 2 == 1, "kernel_size must be odd when strides = 1"
        if self.strides == 2:
            assert self.kernel_size % 2 == 0, "kernel_size must be even when strides = 2"
        
        self.kernel_prototype = windowed_circular_lowpass_kernel(cutoff, self.kernel_size)
        
    def build(self, input_shape):
        self.filtering = tf.keras.layers.DepthwiseConv2D(self.kernel_size, trainable=False, use_bias=False, padding='same', strides=self.strides, weights=[channeled_kernel(self.kernel_prototype, input_shape[-1])])

    def call(self, inputs):
        return self.filtering(inputs)

def get_upsample_multiplication_tensor(input_shape):
    tile = tf.constant([[0, 0], [0, 4]], dtype=tf.float32)
    return tf.reshape(tf.tile(tile, [input_shape[-3], input_shape[-2]]), (1, input_shape[-3]*2, input_shape[-2]*2, 1))

class IsotropicLowpassFilteredUpsampleLayer(tf.keras.layers.Layer):
    def __init__(self, cutoff, kernel_size):
        super(IsotropicLowpassFilteredUpsampleLayer, self).__init__()
        self.kernel_size = kernel_size
        assert self.kernel_size % 2 == 0, "kernel_size must be even"
        self.upsampling = tf.keras.layers.UpSampling2D(size = (2, 2), interpolation='nearest')
        self.multiply = tf.keras.layers.Multiply()
        #self.zeropadding = tf.keras.layers.ZeroPadding2D(padding = ((0, 1), (0, 1)))  # not needed for input even -> output even
        self.kernel_prototype = windowed_circular_lowpass_kernel(cutoff, kernel_size)
        self.kernel_size = kernel_size
        
    def build(self, input_shape):
        self.multiplication_tensor = get_upsample_multiplication_tensor(input_shape)
        self.filtering = tf.keras.layers.DepthwiseConv2D(self.kernel_size, trainable=False, use_bias=False, padding='same', weights=[channeled_kernel(self.kernel_prototype, input_shape[-1])])

    def call(self, inputs):
        x = self.upsampling(inputs)
        x = self.multiply([x, self.multiplication_tensor])
        #x = self.zeropadding(x)  # not needed for input even -> output even
        x = self.filtering(x)
        return x    
    
print("Test input:")
input_shapes = (aa_odd_kernel_size + 2, aa_odd_kernel_size + 2, 3)
print(input_shapes)
eye = tf.repeat(tf.expand_dims(tf.repeat(tf.expand_dims(tf.eye(input_shapes[0], input_shapes[1]), -1), repeats = input_shapes[2], axis = -1), 0), repeats = batch_size, axis = 0)
print(eye)
plt.imshow(eye[0])
plt.show()

def TestModel(input_shapes):
    inputs = tf.keras.Input(input_shapes)
    outputs = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(inputs)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)    
    return model

print("Isotropic lowpass filtered:")
model = TestModel(input_shapes)
output = model(eye)
plt.imshow(output[0])
plt.show()

def TestModel(input_shapes):
    inputs = tf.keras.Input(input_shapes)
    outputs = IsotropicLowpassFilteredUpsampleLayer(np.pi/oversampling, aa_even_kernel_size)(inputs)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)    
    return model

print("Isotropic lowpass filtered upsampled:")
model = TestModel(input_shapes)
output = model(eye)
plt.imshow(output[0])
plt.show()

## Get a batch from the dataset by recropping photos
This should be done in the beginning of each epoch by calling RandomCropper.recrop()

In [ ]:
class RandomCropper:
    def __init__(self, dataset, first_seed = None):
        if first_seed is None:
            first_seed = int(str(int((datetime.utcnow() - datetime(1970, 1, 1)).total_seconds()*1_000_000))[::-1][:16])
        self.dataset_seeded = dataset.enumerate(first_seed)
    
    def recrop(self, size):
        cropped_dataset = self.dataset_seeded.map(lambda seed, image: tf.image.stateless_random_crop(image, (size, size, 3), [seed, 0]), num_parallel_calls=tf.data.experimental.AUTOTUNE, deterministic=False)
        self.dataset_seeded = self.dataset_seeded.map(lambda seed, image: (seed + self.dataset_seeded.cardinality(), image), num_parallel_calls=tf.data.experimental.AUTOTUNE, deterministic=False)
        return cropped_dataset
    
train_dataset_cropper = RandomCropper(train_dataset)
print("First recrop, first item:")
iterator = iter(train_dataset_cropper.recrop(train_image_cropped_size))
plt.imshow((next(iterator) + 1)/2)
plt.show()
print("Second recrop, first item:")
iterator = iter(train_dataset_cropper.recrop(train_image_cropped_size))
plt.imshow((next(iterator) + 1)/2)
plt.show()
print("Second recrop, second item:")
plt.imshow((next(iterator) + 1)/2)
plt.show()

## Generator model
Define the generator network, parameterized by input image size.

In [ ]:
def Generator(input_size, nonlinearity, initializer):      
    inputs = tf.keras.Input(shape = (input_size, input_size, 1), name = "input")
  
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(inputs)
  
    #-----------------First down

    x = tf.keras.layers.Conv2D(8, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
    x = tf.keras.layers.Activation(nonlinearity)(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)
    x = tf.keras.layers.Conv2D(8, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
    x = tf.keras.layers.Activation(nonlinearity)(x)
    x = tf.keras.layers.LeakyReLU()(x)
    # Two activations, huh?
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_even_kernel_size, strides=2)(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)

    #-----------------Second down

    x = tf.keras.layers.Conv2D(16, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(nonlinearity)(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)
    x = tf.keras.layers.Conv2D(16, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(nonlinearity)(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_even_kernel_size, strides=2)(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)
    
    #-----------------Third down

    x = tf.keras.layers.Conv2D(32, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(nonlinearity)(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)
    x = tf.keras.layers.Conv2D(32, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(nonlinearity)(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_even_kernel_size, strides=2)(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)

    #----------------- Fourth down

    x = tf.keras.layers.Conv2D(64, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(nonlinearity)(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)
    x = tf.keras.layers.Conv2D(64, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(nonlinearity)(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_even_kernel_size, strides=2)(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)

    #----------------- Fifth down

    x = tf.keras.layers.Conv2D(128, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(nonlinearity)(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)
    x = tf.keras.layers.Conv2D(128, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(nonlinearity)(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_even_kernel_size, strides=2)(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)

    #----------------- Fifth last up

    x = IsotropicLowpassFilteredUpsampleLayer(np.pi/oversampling, aa_even_kernel_size)(x)
    x = tf.keras.layers.Conv2D(64, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(nonlinearity)(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)
    x = tf.keras.layers.Conv2D(64, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(nonlinearity)(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)    

    #----------------- Fourth last up

    x = IsotropicLowpassFilteredUpsampleLayer(np.pi/oversampling, aa_even_kernel_size)(x)
    x = tf.keras.layers.Conv2D(32, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(nonlinearity)(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)
    x = tf.keras.layers.Conv2D(32, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(nonlinearity)(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)    
    
    #----------------- Third last up

    x = IsotropicLowpassFilteredUpsampleLayer(np.pi/oversampling, aa_even_kernel_size)(x)
    x = tf.keras.layers.Conv2D(16, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(nonlinearity)(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)
    x = tf.keras.layers.Conv2D(16, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(nonlinearity)(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)    
    
    #----------------- Second last up

    x = IsotropicLowpassFilteredUpsampleLayer(np.pi/oversampling, aa_even_kernel_size)(x)
    x = tf.keras.layers.Conv2D(8, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(nonlinearity)(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)
    x = tf.keras.layers.Conv2D(8, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(nonlinearity)(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)

    #-----------------Last up

    x = IsotropicLowpassFilteredUpsampleLayer(np.pi/oversampling, aa_even_kernel_size)(x)
    x = tf.keras.layers.Conv2D(3, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
    x = tf.keras.layers.Activation(nonlinearity)(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)
    x = tf.keras.layers.Conv2D(3, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
    x = tf.keras.layers.Activation("tanh")(x)
    x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)
    
    # Crop the center
    
    #x = tf.keras.layers.CenterCrop(input_size//2, input_size//2)(x)

    return  tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
input_size = train_image_cropped_size
initializer = tf.keras.initializers.RandomUniform(-0.1, 0.1)
generator = Generator(input_size, "relu", "glorot_uniform")
generator.build((4, input_size, input_size, 1))
#Added model diagram
tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)

In [ ]:
# Generate, normalize for display, again
generator_output = generator(tf.random.uniform((4, input_size, input_size, 1), minval=0, maxval=None), training = True)[0].numpy()
print(np.amin(generator_output, axis = (0, 1)))
print(np.amax(generator_output, axis = (0, 1)))
generator_output -= np.amin(generator_output, axis = (0, 1))
generator_output /= np.amax(generator_output, axis = (0, 1))
plt.imshow(generator_output)
plt.show()

# Generate, display
generator_output = generator(tf.random.uniform((4, input_size, input_size, 1), minval=0, maxval=None), training = True)[0].numpy()
plt.imshow(generator_output)
plt.show()

# Generate, normalize for display, again
generator_output = generator(tf.random.uniform((4, input_size, input_size, 1), minval=0, maxval=None), training = True)[0].numpy()
generator_output -= np.amin(generator_output, axis = (0, 1))
generator_output /= np.amax(generator_output, axis = (0, 1))
plt.imshow(generator_output)
plt.show()

In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def generator_loss(disc_generated_output, gen_output):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)
  return gan_loss

In [ ]:
def Discriminator(input_size, nonlinearity, initializer):
  
  tar = tf.keras.layers.Input(shape=[input_size, input_size, 3], name='input')
  
  x = tar
  
  x = tf.keras.layers.Conv2D(8, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
  x = tf.keras.layers.Activation(nonlinearity)(x)
  x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)
  x = tf.keras.layers.Conv2D(8, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
  x = tf.keras.layers.Activation(nonlinearity)(x)
  x = tf.keras.layers.LeakyReLU()(x)
  # Two activations, huh?
  x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_even_kernel_size, strides=2)(x)
  x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)
  
  x = tf.keras.layers.Conv2D(16, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation(nonlinearity)(x)
  x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)
  x = tf.keras.layers.Conv2D(16, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation(nonlinearity)(x)
  x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_even_kernel_size, strides=2)(x)
  x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)
  
  x = tf.keras.layers.Conv2D(32, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation(nonlinearity)(x)
  x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)
  x = tf.keras.layers.Conv2D(32, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation(nonlinearity)(x)
  x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_even_kernel_size, strides=2)(x)
  x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)
  
  x = tf.keras.layers.Conv2D(64, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation(nonlinearity)(x)
  x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)
  x = tf.keras.layers.Conv2D(64, 5, kernel_initializer=initializer, use_bias=True, padding="same")(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation(nonlinearity)(x)
  x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size, strides=1)(x)
  x = IsotropicLowpassFilterLayer(np.pi/oversampling, aa_odd_kernel_size)(x)
  
  x = tf.keras.layers.Conv2D(1, 4, strides=1, kernel_initializer=initializer)(x)  # (bs, 30, 30, 1)

  return tf.keras.Model(inputs=tar, outputs=x)

In [ ]:
initializer = tf.keras.initializers.RandomUniform(-0.1, 0.1)
discriminator = Discriminator(input_size, "relu", "glorot_uniform")
discriminator.build((batch_size, input_size, input_size, 3))
#Added model diagram
tf.keras.utils.plot_model(discriminator, show_shapes=True, dpi=64)

In [ ]:
disc_out = discriminator(generator_output[tf.newaxis, ...], training=True)
plt.imshow(disc_out[0, ..., -1], cmap='RdBu_r')
plt.colorbar()

In [ ]:
def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss

In [ ]:
def select_optimizer():
  #optimizer = tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.5, beta_2=0.99, epsilon=1e-07)
  optimizer = tf.keras.optimizers.Adam()
  return optimizer

generator_optimizer = select_optimizer()
discriminator_optimizer = select_optimizer()

In [ ]:
#The structure for batching the randomly cropped elements
dataset = train_dataset_cropper.recrop(train_image_cropped_size)
tensor_list = []
for i in dataset.take(-1):
  tensor_list.append(i)
tensor_list = tf.convert_to_tensor(tensor_list)

In [ ]:
save_every = 100 # How many epochs to skip between data display and saving

checkpoint_dir = './training_checkpoints_gan'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator=generator,
                                 discriminator=discriminator)
checkpoint.save(file_prefix=checkpoint_prefix)

def checkpoint_save(epoch, inEpochs = 100):
  if (epoch + 1) % inEpochs == 0:
    checkpoint.save(file_prefix=checkpoint_prefix)

def generate_images(generator, save = False, epoch = 0): # the training parameter is for batchnorm, in case it is used
  z = tf.random.uniform((batch_size, input_size, input_size, 1), minval=0, maxval=None)
  prediction = generator(z, training=True)
  plt.figure(figsize = (7, 7))
  first = prediction[0, ...]
  display_list = [first]
  title = ['Network output']
  
  for i in range(1):
    plt.subplot(1, 1, i+1)
    plt.title(title[i])
    plt.imshow(display_list[i])
    plt.axis('off')
  plt.show()
  
@tf.function
def train_step(z, target, epoch):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator(z, training=True)

    disc_real_output = discriminator(target, training=True)
    disc_generated_output = discriminator(gen_output, training=True)

    gen_gan_loss = generator_loss(disc_generated_output, gen_output)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)
    
    generator_gradients = gen_tape.gradient(gen_gan_loss,
                                          generator.trainable_variables)
    discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)
    
  
    generator_optimizer.apply_gradients(zip(generator_gradients,
                                            generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                                discriminator.trainable_variables))

  with summary_writer.as_default():
    tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=epoch)
    tf.summary.scalar('disc_loss', disc_loss, step=epoch)

def fit(epochs, dataset):
  for epoch in range(epochs + 1):
    checkpoint_save(epoch)
    # Train
    start = time.time()
    if epoch > 0:
      z = tf.random.uniform((batch_size, input_size, input_size, 1), minval=0, maxval=None)
      dataset = train_dataset_cropper.recrop(train_image_cropped_size)
      tensor_list = []
      for i in dataset.take(-1):
        tensor_list.append(i)
      target = tf.convert_to_tensor(tensor_list)
      train_step(z, target, epoch)
      #display.clear_output(wait=True)
    elapsed = time.time() - start
    for target in dataset.take(1):
      generate_images(generator) 
       #training = ?
      if epoch == 0:
          print("....")
      else:
          print('Training epoch {}, took {} s\n'.format(epoch, elapsed))

  checkpoint.save(file_prefix=checkpoint_prefix)

In [ ]:
for target in dataset.take(1):
  generate_images(generator)

In [ ]:
import datetime
log_dir = "logs/"
im_dir = "images/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
os.makedirs(im_dir)
summary_writer = tf.summary.create_file_writer(
  log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir {log_dir} --host localhost --port 6221

In [ ]:
EPOCHS = 10000
fit(EPOCHS, dataset)